# En este Jupyter Notebook se realiza la EDA y limpieza del conjunto de datos train para el posterior trabajo de prospeccion de modelos de Machine Learning

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder , OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import seaborn as sns

In [3]:
bnb = pd.read_csv("../data/train.csv")



In [4]:
pd.set_option('display.max_columns', None)
bnb.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,15148661,4.499810,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,flexible,True,NYC,A great spacious apartment in an awesome and c...,2016-07-25,t,t,80%,2015-11-04,f,2017-09-10,40.757349,-73.921339,Sunny Spacious Apt in Astoria w/ back yard access,Astoria,9,96.0,https://a0.muscache.com/im/pictures/3e40924b-7...,11101,1.0,1.0
1,1227528,3.970292,Apartment,Private room,"{Internet,""Wireless Internet"",""Air conditionin...",2,1.0,Futon,strict,True,NYC,Newly Renovated Two Bedroom Apartment in Hamil...,2016-09-01,t,t,NaN,2014-08-26,f,2017-01-05,40.823556,-73.954573,Manhattan Sunny Private Room,Hamilton Heights,8,100.0,https://a0.muscache.com/im/pictures/2953dddd-0...,10031,1.0,1.0
2,15692740,4.787492,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",""Wheel...",4,1.0,Real Bed,moderate,True,NYC,"One bedroom, one bathroom apartment, with an o...",2016-01-05,t,t,100%,2011-01-29,f,2017-09-27,40.689306,-73.955478,Sweet Apartment on Quiet Block,Bedford-Stuyvesant,12,100.0,NaN,11205,1.0,1.0
3,19115685,4.700480,Apartment,Entire home/apt,"{Internet,""Wireless Internet"",""Air conditionin...",3,NaN,Real Bed,moderate,False,NYC,"Comfy, art- and book-filled one-bedroom apartm...",2014-01-06,t,f,NaN,2011-10-26,f,2014-01-13,40.681965,-73.965490,Husband & Wife Art-Filled Apartment,Clinton Hill,2,90.0,https://a0.muscache.com/im/pictures/81766960/b...,11238.0,1.0,1.0
4,2554110,5.192957,Apartment,Entire home/apt,"{TV,""Wireless Internet"",""Air conditioning"",Kit...",3,1.0,Real Bed,strict,True,NYC,1br/1ba. Spacious living room with plenty of s...,2016-05-20,t,f,NaN,2012-06-21,f,2016-05-20,40.742842,-73.957554,Luxurious 1 Bedroom Apt!,Long Island City,1,80.0,https://a0.muscache.com/im/pictures/0ed6ee9e-6...,11101,1.0,1.0


In [5]:
bnb.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59288 entries, 0 to 59287
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      59288 non-null  int64  
 1   log_price               59288 non-null  float64
 2   property_type           59288 non-null  object 
 3   room_type               59288 non-null  object 
 4   amenities               59288 non-null  object 
 5   accommodates            59288 non-null  int64  
 6   bathrooms               59139 non-null  float64
 7   bed_type                59288 non-null  object 
 8   cancellation_policy     59288 non-null  object 
 9   cleaning_fee            59288 non-null  bool   
 10  city                    59288 non-null  object 
 11  description             59288 non-null  object 
 12  first_review            46601 non-null  object 
 13  host_has_profile_pic    59148 non-null  object 
 14  host_identity_verified  59148 non-null

# Convierto todos los property_type que aparecen menos de 10 veces por "Other"

In [6]:
prop_counts = bnb['property_type'].value_counts()
prop_counts

Apartment             39158
House                 13295
Condominium            2103
Townhouse              1341
Loft                    992
Other                   469
Guesthouse              401
Bed & Breakfast         372
Bungalow                297
Villa                   142
Dorm                    106
Guest suite              99
Camper/RV                80
In-law                   62
Hostel                   61
Timeshare                59
Cabin                    58
Boutique hotel           57
Boat                     49
Serviced apartment       17
Tent                     15
Castle                   11
Vacation home            10
Yurt                      7
Hut                       6
Treehouse                 5
Earth House               4
Tipi                      3
Cave                      2
Chalet                    2
Train                     2
Lighthouse                1
Casa particular           1
Island                    1
Name: property_type, dtype: int64

In [7]:
# Identificar tipos con recuento menor que 10
tipos_a_cambiar = prop_counts[prop_counts < 10].index

# Cambiar los valores en la columna property_type
bnb['property_type'] = bnb['property_type'].map(lambda x: 'Other' if x in tipos_a_cambiar else x)


# Genero una variable mean_log_price como la media del log_price de los pares property_type - accommodates

In [8]:
# Supongamos que 'bnb' es tu DataFrame
# Calcula la media del log_price para cada combinación de property_type y accommodates
grouped_df = bnb.groupby(['property_type','accommodates'])['log_price'].mean().reset_index()

# Renombra la columna resultante
grouped_df = grouped_df.rename(columns={'log_price': 'mean_log_price'})

# Fusiona los resultados de nuevo en el DataFrame original
bnb = pd.merge(bnb, grouped_df, on=['property_type','accommodates'], how='left')

# Relleno los valores nulos en la columna bathroom con el promedio de baños para el número de huespedes 

In [10]:
# Calcular el promedio de baños para cada número de acomodates
average_bathrooms = bnb.groupby('accommodates')['bathrooms'].transform('mean')

# Llenar los valores nulos en la columna 'bathrooms' con los promedios calculados
bnb['bathrooms'] = bnb['bathrooms'].fillna(average_bathrooms)
bnb['bathrooms'] = bnb['bathrooms'].round(1)

# Paso la columna cleaning_fee de tipo bool a int

In [11]:
bnb['cleaning_fee'] = bnb['cleaning_fee'].astype(int)

# Parece que no hay gran diferencia entre tener foto de perfil o estar verificado en el log_price por lo que se van a eliminar esas dos columnas


In [12]:
bnb[bnb.host_has_profile_pic == "t"].log_price.mean() , bnb[bnb.host_has_profile_pic == "f"].log_price.mean()

(4.782476060710248, 4.9516436799328964)

In [13]:
bnb[bnb.host_identity_verified == "t"].log_price.mean() , bnb[bnb.host_identity_verified == "f"].log_price.mean()

(4.79577355988958, 4.756846320603111)

In [14]:
columnas_a_eliminar = ['host_has_profile_pic', 'host_identity_verified']
bnb = bnb.drop(columnas_a_eliminar, axis=1)

# Relleno los nulos de la columna host_since por el valor de first_review cuando sea posible, y por la fecha más reciente de host_since cuando first_review sea nulo tambien

In [15]:
bnb['host_since'] = bnb['host_since'].fillna(bnb['first_review'])

In [16]:
bnb['host_since'] = bnb['host_since'].fillna("-")

In [17]:
bnb['host_since'] = bnb['host_since'].replace('-', '2017-10-04')

In [18]:
bnb = bnb.drop("first_review", axis=1)

# Convierto a int la columna host_response_rate y relleno con la media los nulos

In [19]:
bnb['host_response_rate'] = bnb['host_response_rate'].str.rstrip('%').astype('float')

# Rellenar los valores nulos con la media
media = bnb['host_response_rate'].mean()
bnb['host_response_rate'].fillna(media, inplace=True)


In [20]:
bnb['host_response_rate'] = bnb['host_response_rate'].astype(int)

# Cambio la columna instant_bookable a binario

In [21]:
bnb['instant_bookable'] = bnb['instant_bookable'].replace('f', 0)
bnb['instant_bookable'] = bnb['instant_bookable'].replace('t', 1)

# Elimino la columna last_review y las columnas latitude y longitude ya que posteriormente voy a tratar de cuantificar la ubicación del alojamiento mediante clusterización de neighbourhood

In [22]:
bnb = bnb.drop("last_review", axis=1)


In [23]:
len(bnb[bnb.neighbourhood.isna()])

5523

# Utilizo el modelo NearestNeighbors para asignar un barrio a aquellos valores nulos en función de las columnas ['latitude', 'longitude']

In [24]:
bnb_nulos = bnb[bnb['neighbourhood'].isnull()]

# Filtrar las filas sin valores nulos en 'barrio'
bnb_no_nulos = bnb.dropna(subset=['neighbourhood'])

# Definir el número de vecinos cercanos a considerar
num_neighbors = 1

# Inicializar el modelo NearestNeighbors
neighbors_model = NearestNeighbors(n_neighbors=num_neighbors)

# Ajustar el modelo a las coordenadas sin valores nulos en 'barrio'
neighbors_model.fit(bnb_no_nulos[['latitude', 'longitude']])

# Encontrar los vecinos más cercanos para las filas con valores nulos en 'barrio'
distances, indices = neighbors_model.kneighbors(bnb_nulos[['latitude', 'longitude']])

# Asociar los valores nulos en 'barrio' con los barrios correspondientes de los vecinos más cercanos
bnb.loc[bnb['neighbourhood'].isnull(), 'neighbourhood'] = bnb_no_nulos.iloc[indices[:, 0]]['neighbourhood'].values


# Se va a utilizar el barrio como variable de localización, por lo que se elimina la columna zipcode, además de la thumbnail_url, ya que escapa al análisis

In [25]:
bnb = bnb.drop("thumbnail_url", axis=1)
bnb = bnb.drop("zipcode", axis=1)

# Se rellenan los nulos de review_scores_rating con la media

In [26]:
bnb['review_scores_rating'] = bnb['review_scores_rating'].fillna(bnb.review_scores_rating.mean())
bnb['review_scores_rating'] = bnb['review_scores_rating'].astype(int)

# Relleno los valores nulos en la columna bedrooms con el promedio de baños para el número de huespedes 

In [27]:
average_bedrooms = bnb.groupby('accommodates')['bedrooms'].transform('mean')

# Llenar los valores nulos en la columna 'bathrooms' con los promedios calculados
bnb['bedrooms'] = bnb['bedrooms'].fillna(average_bedrooms)
bnb['bedrooms'] = bnb['bedrooms'].round(1)

In [28]:
bnb['bedrooms'] = bnb['bedrooms'].astype(int)

# Se eliminan las columnas name y amenities por escapar del alcande de este análisis

In [29]:
bnb = bnb.drop("name", axis=1)
bnb = bnb.drop("amenities", axis=1)

# Relleno los valores nulos en la columna beds con el promedio de baños para el número de huespedes

In [30]:
average_bedrooms = bnb.groupby('accommodates')['beds'].transform('mean')

# Llenar los valores nulos en la columna 'bathrooms' con los promedios calculados
bnb['beds'] = bnb['beds'].fillna(average_bedrooms)
bnb['beds'] = bnb['beds'].round(1)
bnb['beds'] = bnb['beds'].astype(int)

# Utilizo la columna host_since para categorizar la antiguedad del host en 4 categorías

In [31]:
# Convertir la columna 'host_since' a tipo datetime
bnb['host_since'] = pd.to_datetime(bnb['host_since'])

# 1. Extracción de Año y Mes
bnb['host_year'] = bnb['host_since'].dt.year
bnb['host_month'] = bnb['host_since'].dt.month

# 2. Calcular la Antigüedad en Años
bnb['host_age_years'] = (pd.to_datetime('2017-10-04 00:00:00') - bnb['host_since']).dt.days / 365.25

# 3. Creación de Categorías por Rangos de Tiempo
bins = [0, 1, 2, 5, float('inf')]  # Ajustar los puntos de corte según el rango de fechas
labels = ['nuevo', 'poco_experimentado', 'experimentado', 'muy_experimentado']
bnb['host_experience'] = pd.cut(bnb['host_age_years'], bins=bins, labels=labels)

In [32]:
bnb.host_experience.value_counts()

experimentado         30893
poco_experimentado    11583
muy_experimentado     10839
nuevo                  5938
Name: host_experience, dtype: int64

In [33]:
columns = ['host_year', 'host_month']
bnb = bnb.drop(columns, axis=1)

In [34]:
len(bnb.neighbourhood.value_counts())

611

# Voy a clusterizar los barrios en función de log_price 

In [36]:
# Asumiendo que 'log_price' y 'neighbourhood' son columnas relevantes en tu DataFrame
X = bnb[['log_price', 'neighbourhood']]

# Calcula la media del log_price para cada barrio
neighbourhood_means = X.groupby('neighbourhood')['log_price'].mean().reset_index()

# Crea un DataFrame con las medias del log_price por barrio
neighbourhood_means.columns = ['neighbourhood', 'log_price_mean']

# Normaliza la columna 'log_price_mean'
scaler = StandardScaler()
neighbourhood_means_scaled = scaler.fit_transform(neighbourhood_means[['log_price_mean']])

# Aplica el algoritmo de k-means a las medias normalizadas
kmeans = KMeans(n_clusters=12, random_state=42)
neighbourhood_means['neighbourhood_cluster'] = kmeans.fit_predict(neighbourhood_means_scaled)

# Ordena los clusters según la media del log_price
cluster_order = neighbourhood_means.groupby('neighbourhood_cluster')['log_price_mean'].mean().sort_values().index
neighbourhood_means['neighbourhood_cluster'] = neighbourhood_means['neighbourhood_cluster'].map(
    {cluster: order + 1 for order, cluster in enumerate(cluster_order)}
)

# Crea un diccionario de mapeo de barrios a clusters
neighbourhood_cluster_mapping = dict(zip(neighbourhood_means['neighbourhood'], neighbourhood_means['neighbourhood_cluster']))

# Aplica el mapeo de clusters al DataFrame original
bnb['neighbourhood_cluster'] = bnb['neighbourhood'].map(neighbourhood_cluster_mapping)

C:\Users\daarr\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\daarr\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


In [37]:
# Grafica los puntos de los barrios y sus clusters, coloreando por property_type
plt.figure(figsize=(12, 8))
sns.scatterplot(x='log_price', y='neighbourhood', hue='neighbourhood_cluster',
                style='neighbourhood', data=bnb, palette='viridis', s=100)
plt.title('Clusters de Tipos de Propiedad basados en Media de Log_Price')
plt.xlabel('Log_Price')
plt.ylabel('Barrio')
plt.legend(title='Cluster', loc='upper right', bbox_to_anchor=(1.25, 1))
plt.show();

In [39]:
bnb = bnb.drop("latitude", axis=1)
bnb = bnb.drop("longitude", axis=1)

In [41]:
bnb['host_experience'] = bnb['host_experience'].fillna("nuevo")

In [42]:
columns = ["host_age_years","host_since"]
bnb = bnb.drop(columns, axis=1)

In [46]:
bnb.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59288 entries, 0 to 59287
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   id                     59288 non-null  int64   
 1   log_price              59288 non-null  float64 
 2   property_type          59288 non-null  object  
 3   room_type              59288 non-null  object  
 4   accommodates           59288 non-null  int64   
 5   bathrooms              59288 non-null  float64 
 6   bed_type               59288 non-null  object  
 7   cancellation_policy    59288 non-null  object  
 8   cleaning_fee           59288 non-null  int32   
 9   city                   59288 non-null  object  
 10  description            59288 non-null  object  
 11  host_response_rate     59288 non-null  int32   
 12  instant_bookable       59288 non-null  int64   
 13  neighbourhood          59288 non-null  object  
 14  number_of_reviews      59288 non-null 

In [47]:
bnb.head()

,id,log_price,property_type,room_type,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,host_response_rate,instant_bookable,neighbourhood,number_of_reviews,review_scores_rating,bedrooms,beds,mean_log_price,host_experience,neighbourhood_cluster
0,15148661,4.499810,Apartment,Entire home/apt,2,1.0,Real Bed,flexible,1,NYC,A great spacious apartment in an awesome and c...,80,0,Astoria,9,96,1,1,4.594133,poco_experimentado,6
1,1227528,3.970292,Apartment,Private room,2,1.0,Futon,strict,1,NYC,Newly Renovated Two Bedroom Apartment in Hamil...,94,0,Hamilton Heights,8,100,1,1,4.594133,experimentado,5
2,15692740,4.787492,Apartment,Entire home/apt,4,1.0,Real Bed,moderate,1,NYC,"One bedroom, one bathroom apartment, with an o...",100,0,Bedford-Stuyvesant,12,100,1,1,5.072061,muy_experimentado,5
3,19115685,4.700480,Apartment,Entire home/apt,3,1.1,Real Bed,moderate,0,NYC,"Comfy, art- and book-filled one-bedroom apartm...",94,0,Clinton Hill,2,90,1,1,4.861277,muy_experimentado,7
4,2554110,5.192957,Apartment,Entire home/apt,3,1.0,Real Bed,strict,1,NYC,1br/1ba. Spacious living room with plenty of s...,94,0,Long Island City,1,80,1,1,4.861277,muy_experimentado,7


In [48]:
bnb.columns

Index(['id', 'log_price', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bed_type', 'cancellation_policy', 'cleaning_fee', 'city',
       'description', 'host_response_rate', 'instant_bookable',
       'neighbourhood', 'number_of_reviews', 'review_scores_rating',
       'bedrooms', 'beds', 'mean_log_price', 'host_experience',
       'neighbourhood_cluster'],
      dtype='object')

In [49]:
bnb = bnb[['log_price', 'room_type', 'cleaning_fee',
           'cancellation_policy','city','host_experience',
           'accommodates','bathrooms', 'bedrooms', 'property_type','mean_log_price','neighbourhood_cluster','neighbourhood']]

In [50]:
bnb.to_csv("../data/train_clean3.csv", index=False)